## Project: Wildfire Destructive Potential Index 

### Module 1: *Data Processing*

---
### Contents  
- 1. *Data Loading and Exploration* 
- 2. *Data Cleaning and Merging*
- 3. *Data Imputation*
- 3. *File Export*
---
### Notes

**Clean and process daily weather readings from California CIMIS irrigation stations (2018-01-01 to 2020-09-18). Integrate wildfire impact data from the same period, including structures damaged/destroyed, fatalities, and acres burned. Also include weather readings related to the following incidents for case study:**

- **Palisades and Eaton Fires** (2025-01-07)\
        Palisades - *High damage*, 7810 structures damaged or destroyed, 13 fatalities\
        Eaton Fire - *High damage* 10,488 structures damaged or destroyed, 27 fatalities, 14,021 acres burned
- **Dixie Fire** (2021-07-13)\
        *High damage*, 963,309 acres burned, 1405 structures damaged or destroyed, 1 fatality.


---
### Inputs
*Raw Data*
- Weather Data - `all_conditions.csv`, `january 2025.csv`, `july 2021.csv`
- Fire Data - `damagesv2.csv`
- Miscellaneous Data - `i04_CIMIS_Weather_Stations.csv`, `pop_data`, `mean_income` 
---
### Outputs  
- `weasther_fire_pop_income.csv` Cleaned weather dataset merged with fire damage severity 
- `palisades_weather.csv`,`dixie_weather.csv` Cleaned waether datasets for case studies
---
### User Created Dependencies  

In [1]:
import os
import sys

# Allow import of custom modules from the parent directory
sys.path.append(os.path.abspath(os.path.join('..')))

from src.data_utils import impute_median_data
from src.data_utils import impute_missing_rows
from src.data_utils import data_explore
from src.data_utils import post_merge_check
from src.data_utils import identify_missing_station_dates

---
### Third Party Dependencies

In [2]:
# Data handling
import pandas as pd
import numpy as np

# Geospatial data
from shapely.geometry import Point

# Plotting (for optional EDA or visual validation)
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing and modeling utilities
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

---

## 1. Data Loading and Exploration

### 1.1 CIMIS Historical Weather Data - `all_conditions.csv`

This dataset contains daily weather measurements from CIMIS stations across California. Variables include temperature, precipitation, humidity, solar radiation, and wind.

**California Environmental Conditions Dataset**  
Available at: [https://www.kaggle.com/datasets/thedevastator/california-environmental-conditions](https://www.kaggle.com/datasets/thedevastator/california-environmental-conditions)

This dataset was published by **TheDevastator** and includes CIMIS station-level weather observations across California. All rights and usage guidelines are subject to Kaggle's dataset licensing terms.

In [3]:
weather = pd.read_csv("../data/raw/all_conditions.csv")
weather.loc[:, 'Date'] = pd.to_datetime(weather['Date'], format='mixed')
weather = weather.drop(columns='Target')
weather = data_explore(weather)
weather.head(3)

Duplicates before dropping:  1314 

<class 'pandas.core.frame.DataFrame'>
Index: 126811 entries, 0 to 128124
Data columns (total 18 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Stn Id                126811 non-null  int64  
 1   Stn Name              126811 non-null  object 
 2   CIMIS Region          126811 non-null  object 
 3   Date                  126811 non-null  object 
 4   ETo (in)              126728 non-null  float64
 5   Precip (in)           126811 non-null  float64
 6   Sol Rad (Ly/day)      126811 non-null  float64
 7   Avg Vap Pres (mBars)  126811 non-null  float64
 8   Max Air Temp (F)      126808 non-null  float64
 9   Min Air Temp (F)      126810 non-null  float64
 10  Avg Air Temp (F)      126806 non-null  float64
 11  Max Rel Hum (%)       126811 non-null  float64
 12  Min Rel Hum (%)       126811 non-null  float64
 13  Avg Rel Hum (%)       126798 non-null  float64
 14  Dew Point (F)        

,Stn Id,Stn Name,CIMIS Region,Date,ETo (in),Precip (in),Sol Rad (Ly/day),Avg Vap Pres (mBars),Max Air Temp (F),Min Air Temp (F),Avg Air Temp (F),Max Rel Hum (%),Min Rel Hum (%),Avg Rel Hum (%),Dew Point (F),Avg Wind Speed (mph),Wind Run (miles),Avg Soil Temp (F)
0,2,FivePoints,San Joaquin Valley,2018-01-01 00:00:00,0.06,0.0,219.0,7.3,63.4,35.3,47.8,82.0,46.0,65.0,36.6,3.3,78.3,51.1
1,2,FivePoints,San Joaquin Valley,2018-01-02 00:00:00,0.04,0.0,127.0,7.4,59.8,37.7,47.2,80.0,52.0,67.0,36.7,3.1,74.5,51.3
2,2,FivePoints,San Joaquin Valley,2018-01-03 00:00:00,0.04,0.0,125.0,8.4,61.1,37.3,49.9,79.0,49.0,68.0,39.9,4.5,107.5,51.3


In [4]:
weather['Date'].min()

Timestamp('2018-01-01 00:00:00')

In [5]:
weather['Date'].max()

Timestamp('2020-09-18 00:00:00')

**initial observations:**
- The dataset contains `128,125` rows and `19` columns.
- `Date` column is currently a string object, conversion to date format is neccessary
- various columns contain `NULL` values
- `Target` variable was for risk of fire starting experiment, this will be replaced with a severity index so it is dropped

### 1.2 Wildfire Damage Data - `damagesv2.csv`

Provides severity information from historical recorded wildfires that are used to construct the classification target (low, medium, high) for model training and evaluation. Data was compiled from external team sources and structured for analysis. Relevant variables include:
- `Start` - Date fire started
- `Cont.` - Date fire contained
- `Total` - Total acres burned
- `Dam`,`Dest.` - Structures damaged and destroyed
- `Fire`, `Civil` - Firefighter and Civilian Casualties

Source: [CAL FIRE – Fire Incidents](https://www.fire.ca.gov/)  
This data was manually extracted and structured from publicly available incident records for analytical use.

In [6]:
fire_damage = pd.read_csv("../data/raw/damagesv2.csv")
fire_damage = data_explore(fire_damage)

Duplicates before dropping:  4 

<class 'pandas.core.frame.DataFrame'>
Index: 274 entries, 0 to 277
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Incident #  274 non-null    object
 1   County      274 non-null    object
 2   Fire Name   274 non-null    object
 3   Start       274 non-null    object
 4   Cont.       274 non-null    object
 5   DPA         274 non-null    object
 6   Total       274 non-null    object
 7   Type        274 non-null    object
 8   Cause       274 non-null    object
 9   Dest.       122 non-null    object
 10  Dam.        85 non-null     object
 11  Fire        25 non-null     object
 12  Civil       1 non-null      object
dtypes: object(13)
memory usage: 30.0+ KB


In [7]:
fire_damage.sample(3)

,Incident #,County,Fire Name,Start,Cont.,DPA,Total,Type,Cause,Dest.,Dam.,Fire,Civil
265,SNF-1392*,MERCED,CREEK,09/04/20,12/31/20,USFS,"379,895",T,UNDETERMINED,858,71,NaN,NaN
240,SNF-1160,FRESNO,TRIMMER,08/03/20,09/01/20,USFS,594,"B, T",UNDETERMINED,NaN,NaN,NaN,NaN
228,NOD-3218,LASSEN,R-2,06/23/20,06/25/20,BLM,563,"B, G",UNDETERMINED,NaN,NaN,NaN,NaN


**initial observations:**
- The dataset contains `278` rows and `13` columns.
- all variables are of object type, most will need to be converted to numeric
- `NAN` values present in last 4 columns represent fires with no structural damage or casualties.
- commas in `Total` column may cause issues

### 1.3 Fire Station Metadata - `i04_CIMIS_Weather_Stations.csv`

Contains geographic and descriptive information for all relevant CIMIS weather stations in California. The dataset includes spatial data. including `latitude`, `longitude`, `station ID`, `elevation`, and `regional information`. These are merged with the weather readings to analyze regional weather trends and mapping results.

Source: [California Department of Water Resources – CIMIS](https://cimis.water.ca.gov/)  
Accessed and processed in accordance with public data use policies.

In [8]:
stations = pd.read_csv("../data/raw/i04_CIMIS_Weather_Stations.csv")
stations = data_explore(stations)
stations.head(3)

Duplicates before dropping:  0 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 267 entries, 0 to 266
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   X                     267 non-null    float64
 1   Y                     267 non-null    float64
 2   OBJECTID              267 non-null    int64  
 3   Name                  267 non-null    object 
 4   County                267 non-null    object 
 5   Latitude              267 non-null    float64
 6   Longitude             267 non-null    float64
 7   Status                267 non-null    object 
 8   Date_Data_Refers_To   267 non-null    object 
 9   Comments              0 non-null      float64
 10  Source                267 non-null    object 
 11  Last_Modified_Date    267 non-null    object 
 12  ID                    267 non-null    int64  
 13  DWR_Regional_Office_  267 non-null    object 
 14  Elevation             267 non-null    int

,X,Y,OBJECTID,Name,County,Latitude,Longitude,Status,Date_Data_Refers_To,Comments,Source,Last_Modified_Date,ID,DWR_Regional_Office_,Elevation,Connect,Disconnect,Last_Modified_By
0,-119.732000,36.814000,1,Fresno/F.S.U. USDA,Fresno,36.814000,-119.732000,Inactive,2023/02/08 00:00:00+00,NaN,"CIMIS, DWR",2023/02/09 00:00:00+00,1,SCRO,340,1982/06/07 00:00:00+00,9/25/1988,"AU, GDSS"
1,-120.112906,36.336222,2,Five Points,Fresno,36.336222,-120.112906,Active,2023/02/08 00:00:00+00,NaN,"CIMIS, DWR",2023/02/09 00:00:00+00,2,SCRO,285,1982/06/07 00:00:00+00,Active,"AU, GDSS"
2,-121.793000,36.881000,3,Beach /Santa Cruz CO,Santa Cruz,36.881000,-121.793000,Inactive,2023/02/08 00:00:00+00,NaN,"CIMIS, DWR",2023/02/09 00:00:00+00,3,SCRO,10,1982/05/30 00:00:00+00,8/25/1986,"AU, GDSS"


**initial observations:**
- `OBJECTID` represents station ID to match with CIMIS data
- `County`, `Latitude`, `Longitude`, `Elevation` only variables needed from this dataset and all appear to have zero null values.
- `County` field is different case than damages dataset.

### 1.4 Palisades FIre Weather - `january 2025.csv`

CIMIS weather data extracted from the month of the Palisades fire in California in 2025. This data is used to test the predictive power of the models on fresh data.

Source: [California Department of Water Resources – CIMIS](https://cimis.water.ca.gov/)  
Accessed and processed in accordance with public data use policies.

In [9]:
palisades_weather = pd.read_csv("../data/raw/january 2025.csv")
palisades_weather = palisades_weather.iloc[:-1]
palisades_weather = data_explore(palisades_weather)
palisades_weather.loc[:, 'Stn Id'] = pd.to_numeric(palisades_weather['Stn Id'])
palisades_weather.head(3)

Duplicates before dropping:  0 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4464 entries, 0 to 4463
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Stn Id                4464 non-null   object 
 1   Stn Name              4464 non-null   object 
 2   CIMIS Region          4464 non-null   object 
 3   Date                  4464 non-null   object 
 4   Jul                   4464 non-null   float64
 5   ETo (in)              3881 non-null   float64
 6   Precip (in)           4429 non-null   float64
 7   Sol Rad (Ly/day)      4429 non-null   float64
 8   Avg Vap Pres (mBars)  4429 non-null   float64
 9   Max Air Temp (F)      4429 non-null   float64
 10  Min Air Temp (F)      4429 non-null   float64
 11  Avg Air Temp (F)      4429 non-null   float64
 12  Max Rel Hum (%)       4429 non-null   float64
 13  Min Rel Hum (%)       4429 non-null   float64
 14  Avg Rel Hum (%)       4428 non-null   f

,Stn Id,Stn Name,CIMIS Region,Date,Jul,ETo (in),Precip (in),Sol Rad (Ly/day),Avg Vap Pres (mBars),Max Air Temp (F),Min Air Temp (F),Avg Air Temp (F),Max Rel Hum (%),Min Rel Hum (%),Avg Rel Hum (%),Dew Point (F),Avg Wind Speed (mph),Wind Run (miles),Avg Soil Temp (F)
0,2,FivePoints,San Joaquin Valley,1/1/2025,1.0,0.04,0.0,182.0,8.0,60.2,33.0,45.5,91.0,57.0,77.0,38.7,3.5,83.6,51.0
1,2,FivePoints,San Joaquin Valley,1/2/2025,2.0,0.05,0.0,201.0,8.2,62.3,33.6,46.8,94.0,49.0,76.0,39.5,3.1,73.3,50.7
2,2,FivePoints,San Joaquin Valley,1/3/2025,3.0,0.03,0.0,102.0,10.3,60.1,42.8,49.6,94.0,61.0,85.0,45.2,6.0,144.3,50.8


**initial observations:**
- `Stn Id` converted from object to number for merge

### 1.5 Dixie Fire Weather - `july 2021.csv`

This dataset includes daily weather data from July 2021, corresponding to the Dixie Fire. It is used to evaluate how accurately the model can classify severity based on known conditions.

Source: [California Department of Water Resources – CIMIS](https://cimis.water.ca.gov/)  
Accessed and processed in accordance with public data use policies.

In [10]:
dixie_weather = pd.read_csv("../data/raw/july 2021.csv")
dixie_weather = dixie_weather.iloc[:-1]
dixie_weather = data_explore(dixie_weather)
dixie_weather.loc[:,'Stn Id'] =  pd.to_numeric(dixie_weather['Stn Id'])
dixie_weather.head(3)

Duplicates before dropping:  0 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4448 entries, 0 to 4447
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Stn Id                4448 non-null   object 
 1   Stn Name              4448 non-null   object 
 2   CIMIS Region          4448 non-null   object 
 3   Date                  4448 non-null   object 
 4   Jul                   4448 non-null   float64
 5   ETo (in)              4128 non-null   float64
 6   Precip (in)           4448 non-null   float64
 7   Sol Rad (Ly/day)      4448 non-null   float64
 8   Avg Vap Pres (mBars)  4433 non-null   float64
 9   Max Air Temp (F)      4406 non-null   float64
 10  Min Air Temp (F)      4433 non-null   float64
 11  Avg Air Temp (F)      4445 non-null   float64
 12  Max Rel Hum (%)       4448 non-null   float64
 13  Min Rel Hum (%)       4448 non-null   float64
 14  Avg Rel Hum (%)       4398 non-null   f

,Stn Id,Stn Name,CIMIS Region,Date,Jul,ETo (in),Precip (in),Sol Rad (Ly/day),Avg Vap Pres (mBars),Max Air Temp (F),Min Air Temp (F),Avg Air Temp (F),Max Rel Hum (%),Min Rel Hum (%),Avg Rel Hum (%),Dew Point (F),Avg Wind Speed (mph),Wind Run (miles),Avg Soil Temp (F)
0,2,FivePoints,San Joaquin Valley,7/1/2021,182.0,0.30,0.0,676.0,15.9,96.7,64.5,81.2,69.0,30.0,44.0,57.0,5.1,122.4,80.7
1,2,FivePoints,San Joaquin Valley,7/2/2021,183.0,0.29,0.0,681.0,15.7,100.3,64.3,82.0,67.0,25.0,42.0,56.8,5.0,120.5,80.7
2,2,FivePoints,San Joaquin Valley,7/3/2021,184.0,0.29,0.0,653.0,15.1,99.1,64.5,81.6,67.0,23.0,41.0,55.7,4.9,118.3,80.5


**initial observations:**
- `Stn Id` converted from object to number for merge
---

## 2. Data Cleaning and Merging

 ### 2.1 Historical Weather Merge
 Merge weather data with geographic information of CIMIS stations

In [11]:
# Merge geographic station info into weather_data
weather_station = weather.merge(
    stations[['OBJECTID', 'County', 'Latitude', 'Longitude', 'Elevation']],
    left_on='Stn Id', right_on='OBJECTID',
    how='left', indicator=True
)

In [12]:
post_merge_check(weather_station,weather)
weather_station = weather_station.drop(columns=['_merge'])
display(weather_station.sample(1))

Duplicate Keys:  0
Premerged shape:  (126811, 18)
Merged shape:  (126811, 24)
Duplicates after merge:  0
NA values before merge:  138
NA values after merge:  138


,Stn Id,Stn Name,CIMIS Region,Date,ETo (in),Precip (in),Sol Rad (Ly/day),Avg Vap Pres (mBars),Max Air Temp (F),Min Air Temp (F),...,Avg Rel Hum (%),Dew Point (F),Avg Wind Speed (mph),Wind Run (miles),Avg Soil Temp (F),OBJECTID,County,Latitude,Longitude,Elevation
38205,124,Panoche,San Joaquin Valley,2018-10-01 00:00:00,0.15,0.0,446.0,14.2,86.0,54.2,...,57.0,53.9,2.0,48.7,68.1,124,Fresno,36.890096,-120.731665,183


### 2.2 Palisades Weather Merge
 Merge Palisades fire weather data with geographic information of CIMIS stations

In [13]:
# Merge geographic station info into palisades_weather data
pal_data = palisades_weather.merge(
    stations[['OBJECTID', 'County', 'Latitude', 'Longitude', 'Elevation', 'Name']],
    left_on='Stn Id', right_on='OBJECTID',
    how='left'
)

In [14]:
post_merge_check(pal_data,palisades_weather)

Duplicate Keys:  0
Premerged shape:  (4464, 19)
Merged shape:  (4464, 25)
Duplicates after merge:  0
NA values before merge:  1175
NA values after merge:  1547


The palisades data includes readings from two stations not present in the stations dataset and not matched with county location. These entries are dropped.

In [15]:
pal_data['County'].isna().sum()
no_county_info = pal_data['County'].isna()
pal_data = pal_data[~no_county_info]
pal_data.sample(1)

,Stn Id,Stn Name,CIMIS Region,Date,Jul,ETo (in),Precip (in),Sol Rad (Ly/day),Avg Vap Pres (mBars),Max Air Temp (F),...,Dew Point (F),Avg Wind Speed (mph),Wind Run (miles),Avg Soil Temp (F),OBJECTID,County,Latitude,Longitude,Elevation,Name
2980,217,Moorpark,Central Coast Valleys,1/5/2025,5.0,0.12,0.02,284.0,5.3,79.7,...,28.3,4.4,105.2,60.0,217.0,Ventura,34.269031,-118.849319,718.0,Moorpark


### 2.3 Dixie Weather Merge
Merge Dixie fire weather data with geographic information of CIMIS stations

In [16]:
# Merge geographic station info into dixie_weather data
dixie_data = dixie_weather.merge(
    stations[['OBJECTID', 'County', 'Latitude', 'Longitude', 'Elevation', 'Name']],
    left_on='Stn Id', right_on='OBJECTID',
    how='left'
)

In [17]:
post_merge_check(dixie_data,dixie_weather)
dixie_data.sample(1)

Duplicate Keys:  0
Premerged shape:  (4448, 19)
Merged shape:  (4448, 25)
Duplicates after merge:  0
NA values before merge:  695
NA values after merge:  695


,Stn Id,Stn Name,CIMIS Region,Date,Jul,ETo (in),Precip (in),Sol Rad (Ly/day),Avg Vap Pres (mBars),Max Air Temp (F),...,Dew Point (F),Avg Wind Speed (mph),Wind Run (miles),Avg Soil Temp (F),OBJECTID,County,Latitude,Longitude,Elevation,Name
1743,153,Escondido SPV,South Coast Valleys,7/16/2021,197.0,0.27,0.0,680.0,19.5,93.0,...,62.8,4.7,113.1,86.7,153,San Diego,33.08105,-116.975697,390,Escondido SPV


Format weather data to merge with fire damage dataset

In [18]:
# Format county names to match fire damage dataset
weather_station.loc[:, 'County'] = weather_station['County'].str.upper()

# Convert Date fields to datetime format
weather_station.loc[:, 'Date'] = pd.to_datetime(weather_station['Date'], format='mixed')
pal_data.loc[:, 'Date'] = pd.to_datetime(pal_data['Date'], format='mixed')
dixie_data.loc[:, 'Date'] = pd.to_datetime(dixie_data['Date'], format='mixed')

### 2.4 Clean and Format Fire Damage Dataset

In [19]:
# Backup copy
damage_raw = fire_damage.copy()

# Drop unneeded columns
fire_damage = fire_damage.drop(columns=['Fire Name', 'DPA', 'Incident #','Type','Cause'])

# Convert string values to numeric in relevant columns
cols = ['Total', 'Dest.', 'Dam.', 'Fire', 'Civil']
for col in cols:
    # drop any commas
    fire_damage[col] = fire_damage[col].str.replace(',', '').astype(float)
    fire_damage[col] = pd.to_numeric(fire_damage[col])

# Fill NA values with 0
fire_damage = fire_damage.fillna(0)

# Convert the start and containment dates to datetime
fire_damage['Start'] = pd.to_datetime(fire_damage['Start'], format='mixed')
fire_damage['Cont.'] = pd.to_datetime(fire_damage['Cont.'], format='mixed')

# Fix formatting issue in county names
fire_damage['County'] = fire_damage['County'].str[:-1]

In [20]:
fire_damage.sample(5)

,County,Start,Cont.,Total,Dest.,Dam.,Fire,Civil
42,NAPA,2018-09-08,2018-09-15,2490.0,0.0,0.0,0.0,0.0
258,LASSEN,2020-08-18,2020-09-13,84817.0,1.0,0.0,0.0,0.0
100,TEHAMA,2019-09-05,2019-09-17,8836.0,2.0,0.0,0.0,0.0
240,FRESNO,2020-08-03,2020-09-01,594.0,0.0,0.0,0.0,0.0
64,LASSEN,2018-07-27,2018-08-06,18703.0,0.0,0.0,0.0,0.0


---

## 3. Data Imputation

Display percentage of the amount of NA values in each column

In [21]:
weather_station.isna().mean().round(4) * 100

Stn Id                  0.00
Stn Name                0.00
CIMIS Region            0.00
Date                    0.00
ETo (in)                0.07
Precip (in)             0.00
Sol Rad (Ly/day)        0.00
Avg Vap Pres (mBars)    0.00
Max Air Temp (F)        0.00
Min Air Temp (F)        0.00
Avg Air Temp (F)        0.00
Max Rel Hum (%)         0.00
Min Rel Hum (%)         0.00
Avg Rel Hum (%)         0.01
Dew Point (F)           0.01
Avg Wind Speed (mph)    0.00
Wind Run (miles)        0.00
Avg Soil Temp (F)       0.02
OBJECTID                0.00
County                  0.00
Latitude                0.00
Longitude               0.00
Elevation               0.00
dtype: float64

*impute_median_data* - file from src.data_utils that fills all NA values with the median of each column.

In [22]:
# Fill missing numeric weather values in the data dataframe using the median of each column.
imp_weather_station = impute_median_data(weather_station)
pal_data = impute_median_data(pal_data)
dixie_data = impute_median_data(dixie_data)

In [23]:
imp_weather_station.isna().mean().round(4) * 100

Stn Id                  0.0
Stn Name                0.0
CIMIS Region            0.0
Date                    0.0
ETo (in)                0.0
Precip (in)             0.0
Sol Rad (Ly/day)        0.0
Avg Vap Pres (mBars)    0.0
Max Air Temp (F)        0.0
Min Air Temp (F)        0.0
Avg Air Temp (F)        0.0
Max Rel Hum (%)         0.0
Min Rel Hum (%)         0.0
Avg Rel Hum (%)         0.0
Dew Point (F)           0.0
Avg Wind Speed (mph)    0.0
Wind Run (miles)        0.0
Avg Soil Temp (F)       0.0
OBJECTID                0.0
County                  0.0
Latitude                0.0
Longitude               0.0
Elevation               0.0
dtype: float64

### 3.1 Calculate Fire Duration

Compute the number of days each fire burned based on reported start and containment dates.

In [24]:
# Calculate duration in days
fire_damage['Days'] = fire_damage['Cont.'] - fire_damage['Start']
fire_damage['Days'] = pd.to_timedelta(fire_damage['Days'])
fire_damage['Days'] = fire_damage['Days'].dt.days
fire_damage['Days'] = pd.to_numeric(fire_damage['Days'])

# Ensure minimum duration is at least one day
fire_damage.loc[fire_damage['Days'] == 0, 'Days'] = 1

### 3.2 Drop Stations With Very Few Readings

Some weather stations have data recorded for only a limited number of days, which can introduce high variability in their predictions. Exclude any station that has data for fewer than 900 days.

In [25]:
# Convert the 'Date' column to datetime format (if not already)
station_date = pd.to_datetime(imp_weather_station['Date'])

In [26]:
print("Shape before dropping: ", imp_weather_station.shape)

Shape before dropping:  (126811, 23)


In [27]:
# Count records per station
station_counts = imp_weather_station['Stn Name'].value_counts()

# Identify stations with at least 400 records
reliable_stations = station_counts[station_counts >= 900].index

# Filter the DataFrame to keep only reliable stations
imp_weather_station = imp_weather_station[imp_weather_station['Stn Name'].isin(reliable_stations)]

In [28]:
print("Shape after dropping: ", imp_weather_station.shape)

Shape after dropping:  (99717, 23)


### 3.3 Identify Missing Dates
Some weather stations have gaps in daily records due to maintenance or equipment failures. Loop through all counties and compare the actual recorded dates to the expected continuous date range to identify missing entries.

**Note:** These missing dates could be imputed in future updates.

In [29]:
missing_dates = identify_missing_station_dates(imp_weather_station)

In [30]:
print("Number of missing entries: ", len(missing_dates))

Number of missing entries:  3451


### 3.5 Merge Weather and Fire Damage Datasets

To prepare for feature engineering, we merge the cleaned CIMIS weather data with the fire damage records. This allows us to associate environmental conditions with known fire events, based on shared county and date information.

In [31]:
# Ensure date columns are in datetime format for merge
imp_weather_station['Date'] = pd.to_datetime(imp_weather_station['Date'])
fire_damage['Start'] = pd.to_datetime(fire_damage['Start'])
fire_keep = fire_damage.drop(columns = [ 'Cont.'])

The following changes are made to allow for more than one fire burning per county in a day:
- Damage columns are added together to represent the total damage present in the county. 
- `Days` column represents the amount of days a fire burned for, set this to the max value

In [32]:
agg_dict = {col: 'sum' for col in fire_keep.select_dtypes(include='number').columns}
agg_dict['Days'] = 'max'  # Override 'Days' aggregation

# Group by County and Start with custom aggregation
fire_agg = fire_keep.groupby(['County', 'Start'], as_index=False).agg(agg_dict) 

In [33]:
# Merge weather data with fire damage records
# Matching on county and the start date of the fire
weather_fire = imp_weather_station.merge(
    fire_agg,
    left_on=['County', 'Date'],
    right_on=['County', 'Start'],
    how='left',
    indicator=True
)

# Drop Start column, not needed in future
weather_fire = weather_fire.drop(columns=['Start'])

Since days without fires won’t have matching records in the merge, the damage indicators will appear as NA. Replace these NA values with 0 to prepare for future index calculations.

In [34]:
cols = ['Total','Days','Dam.','Dest.','Fire','Civil']
weather_fire[cols] = weather_fire[cols].fillna(0)

In [35]:
post_merge_check(weather_fire,imp_weather_station)
weather_fire = weather_fire.drop(columns=['_merge'])
weather_fire.sample(2)

Duplicate Keys:  0
Premerged shape:  (99717, 23)
Merged shape:  (99717, 30)
Duplicates after merge:  0
NA values before merge:  0
NA values after merge:  0


,Stn Id,Stn Name,CIMIS Region,Date,ETo (in),Precip (in),Sol Rad (Ly/day),Avg Vap Pres (mBars),Max Air Temp (F),Min Air Temp (F),...,County,Latitude,Longitude,Elevation,Total,Dest.,Dam.,Fire,Civil,Days
5972,202,Nipomo,Central Coast Valleys,2018-03-14,0.05,0.07,232.0,12.2,59.8,47.7,...,SAN LUIS OBISPO,35.028281,-120.560033,255,0.0,0.0,0.0,0.0,0.0,0.0
13115,157,Point San Pedro,San Francisco Bay,2018-05-20,0.16,0.00,557.0,11.5,66.8,49.0,...,MARIN,37.995478,-122.467656,5,0.0,0.0,0.0,0.0,0.0,0.0


## 4. Suplementary Data

### 4.1 Merge population density data

Population Data obtained from 2020 US Census Data.

In [36]:
pop_density = pd.read_csv("../data/raw/pop.csv")

Basic formatting of column names and cases for merging

In [37]:
pop_density.rename(columns={'POPESTIMATE': 'Total Population'}, inplace=True)
pop_keep = pop_density.drop(columns = ['Unnamed: 0','County','YEAR'])
pop_keep.rename(columns={'CTYNAME': 'County'}, inplace=True)
pop_keep['County'] = pop_keep['County'].str.upper()
pop_keep.head()

,County,Total Population,Size (square miles),density,class
0,ALAMEDA,1622188,739.02,2195.052908,urban
1,ALPINE,1141,738.33,1.545379,rural
2,AMADOR,41811,594.58,70.320226,semi-urban
3,BUTTE,207172,1636.46,126.597656,semi-urban
4,CALAVERAS,46565,1020.01,45.651513,semi-urban


Merge in population data to main dataset and case study datasets.

In [38]:
# Adjust case to match other sets
pal_data['County'] = pal_data['County'].str.upper()
dixie_data['County'] = dixie_data['County'].str.upper()

In [39]:
weather_fire_pop = weather_fire.merge(
    pop_keep,
    left_on='County', right_on='County',
    how='left', indicator=True
)

pal_pop = pal_data.merge(
    pop_keep,
    left_on='County', right_on='County',
    how='left', indicator=True
)

dixie_pop = dixie_data.merge(
    pop_keep,
    left_on='County', right_on='County',
    how='left', indicator=True
)

In [40]:
print("Main Weather Data:")
post_merge_check(weather_fire_pop,weather_fire)

print("\nPalisades Weather Data:")
post_merge_check(pal_pop,pal_data)

print("\nDixie Weather Data:")
post_merge_check(dixie_pop,dixie_data)

weather_fire_pop = weather_fire_pop.drop(columns=['_merge'])
pal_pop = pal_pop.drop(columns=['_merge'])
dixie_pop = dixie_pop.drop(columns=['_merge'])

Main Weather Data:
Duplicate Keys:  0
Premerged shape:  (99717, 29)
Merged shape:  (99717, 34)
Duplicates after merge:  0
NA values before merge:  0
NA values after merge:  0

Palisades Weather Data:
Duplicate Keys:  0
Premerged shape:  (4402, 25)
Merged shape:  (4402, 30)
Duplicates after merge:  0
NA values before merge:  0
NA values after merge:  124

Dixie Weather Data:
Duplicate Keys:  0
Premerged shape:  (4448, 25)
Merged shape:  (4448, 30)
Duplicates after merge:  0
NA values before merge:  0
NA values after merge:  128


### 4.2 Merge Mean Income Data

Mean income data obtained from 2020 US Census Data.

In [41]:
mean_income = pd.read_csv("../data/raw/mean_income_by_county.csv")

format data to match case and adjust `Mean Income` column to be numerical

In [42]:
mean_income['County'] = mean_income['County'].str.upper()
mean_income['Mean Income'] = (
    mean_income['Mean Income']
    .str.replace(',', '', regex=False)  # Remove commas
    .astype(int)                        # Convert to integer
)

mean_income.head()

,County,Mean Income
0,ALAMEDA,138489
1,ALPINE,107737
2,AMADOR,85184
3,BUTTE,79233
4,CALAVERAS,84164


Merge mean income data into main dataset and case study data

In [43]:
weather_fire_pop_income = weather_fire_pop.merge(
    mean_income,
    left_on='County', right_on='County',
    how='left', indicator=True
)

pal_pop_income = pal_pop.merge(
    mean_income,
    left_on='County', right_on='County',
    how='left', indicator=True
)

dixie_pop_income = dixie_pop.merge(
    mean_income,
    left_on='County', right_on='County',
    how='left', indicator=True
)

In [44]:
print("Main Weather Data:")
post_merge_check(weather_fire_pop_income,weather_fire_pop)

print("\nPalisades Weather Data:")
post_merge_check(pal_pop_income,pal_pop)

print("\nDixie Weather Data:")
post_merge_check(dixie_pop_income,dixie_pop_income)

Main Weather Data:
Duplicate Keys:  0
Premerged shape:  (99717, 33)
Merged shape:  (99717, 35)
Duplicates after merge:  0
NA values before merge:  0
NA values after merge:  0

Palisades Weather Data:
Duplicate Keys:  0
Premerged shape:  (4402, 29)
Merged shape:  (4402, 31)
Duplicates after merge:  0
NA values before merge:  124
NA values after merge:  155

Dixie Weather Data:
Duplicate Keys:  0
Premerged shape:  (4448, 31)
Merged shape:  (4448, 31)
Duplicates after merge:  0
NA values before merge:  160
NA values after merge:  160


Final Main Dataset

In [45]:
weather_fire_pop_income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99717 entries, 0 to 99716
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Stn Id                99717 non-null  int64         
 1   Stn Name              99717 non-null  object        
 2   CIMIS Region          99717 non-null  object        
 3   Date                  99717 non-null  datetime64[ns]
 4   ETo (in)              99717 non-null  float64       
 5   Precip (in)           99717 non-null  float64       
 6   Sol Rad (Ly/day)      99717 non-null  float64       
 7   Avg Vap Pres (mBars)  99717 non-null  float64       
 8   Max Air Temp (F)      99717 non-null  float64       
 9   Min Air Temp (F)      99717 non-null  float64       
 10  Avg Air Temp (F)      99717 non-null  float64       
 11  Max Rel Hum (%)       99717 non-null  float64       
 12  Min Rel Hum (%)       99717 non-null  float64       
 13  Avg Rel Hum (%) 

---

## 4. Export to file

In [46]:
# Create processed directory if it doesn't exist
os.makedirs("../data/processed", exist_ok=True)

# Save merged and cleaned weather + fire dataset
weather_fire_pop_income.to_csv("../data/processed/merged_weather_fire.csv", index=False)

# Save Palisades weather test data
pal_pop_income.to_csv("../data/processed/palisades_weather.csv", index=False)

# Save Dixie Fire weather test data
dixie_pop_income.to_csv("../data/processed/dixie_weather.csv", index=False)

print("All datasets saved successfully to ../data/processed/")

All datasets saved successfully to ../data/processed/
